# Topic 2. Neural Networks 
## Mathematical background

Credit: The examples and exercises used in this lab are use element from exercises and examples presented in:
        https://github.com/hamarex/tensorflow/blob/master/jupyter_tfbook/Chapter02/Maximum%20likelihood%20estimation%20example.ipynb

We import all the libraries required for the exercises

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import binarize
from sklearn.preprocessing import StandardScaler

We will use the Parkinsons Telemonitoring Data Set available from https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/telemonitoring/
    
    This dataset contains 16 biomedical voice measurements from 42 people with early-stage Parkinson's disease. 
    
    The main aim of the data is to predict the motor and total UPDRS scores ('motor_UPDRS' and 'total_UPDRS') from the 16 voice measures. 

    This can be seen as a regression problem. 

In [ ]:
# Download the dataset  and open using the following commands

# Stores the data as a dataframe of Pandas
df = pd.read_csv('parkinsons_updrs.data')
print(df.columns)

# There are 42 subjects. We will use data for the first one
df_subject_1 = df[df['subject#']==1]
data = df_subject_1.as_matrix()

In [ ]:
# We visualize the relationships between variables in the dataset using seaborn
sns.pairplot(df)
plt.show()

In [ ]:
# We visualize the distribution of the variables, ordered depending the variance they have
order = df.std().sort_values().index
sns.lvplot(data=df, order=order, scale="linear")
plt.show()
# https://seaborn.pydata.org/examples/index.html


In [ ]:
# We scale variables for improved viaulization
df_sca = df - df.min()
df_sca /= df_sca.max()
sns.lvplot(data=df_sca, order=order, scale="linear")
plt.show()

In [ ]:
# From the data, we select the response variable that we are going to model
# It is the total_UPDRS score
target = data[:,5].reshape([len(data), 1])

# The 16 variables that measure the voice will be used as features. 
features = data[:,6:22]

We divide the dataset for the first subject in training and test data. Even rows are in the train set and odd rows in the test set. 




In [ ]:

# Train set 
train_features = features[::2,:]
train_target = target[::2]
train_n_samples = train_features.shape[0]

# Test set
test_features = features[1::2,:]
test_target = target[1::2]
test_n_samples = test_features.shape[0]


In the first analysis we will use only one feature to model the response variable
We select the "best predictor" among the 16 voice features.


In [ ]:
# The SelectKBest function from scikit-learn, with the f_regression metric, is used for feature selection
feature_selector = SelectKBest(f_regression, k=1)
feature_selector.fit(train_features, train_target)

# We print which is the selected feature
print(feature_selector.get_support(), ",", df.columns[6:22][feature_selector.get_support()][0])

We plot the feature and the response variable to inspect the relationship between the two variables visually


In [ ]:
train_sel_feature = feature_selector.transform(train_features)
test_sel_feature = feature_selector.transform(test_features)
plt.plot(train_sel_feature,train_target,'.')
plt.xlabel(df.columns[6:22][feature_selector.get_support()][0])
plt.ylabel('Response variable ')
plt.show()

# Exercise 1

1.1) Complete the missing parameters in the following line to obtain the prediction of the response variable in the test data from a model learned in the train data.

1.2) Check the quality of the prediction by plotting real versus predicted values. This can be done by running the following cell.

1.3) Use l2 regularization as implemented by function Ridge to solve the linear regression problem https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

1.4) Use l1 regularization as implemented by function Lasso https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

Check the quality of the prediction by plotting real versus predicted values. This can be done by running the following cell. 

In [ ]:
predicted_test_target = (LinearRegression().fit()).predict()

In [ ]:
plt.plot(test_sel_feature, test_target, 'b.', label='Original data')
plt.plot(test_sel_feature,predicted_test_target,'vr', label='Scikit-learn LR') 
plt.show()

# Using tensorflow to learn a linear regression model

We will use tensorflow to learn a linear regression model similar to the one learned using scikit-learn (y = Wx + b)

In [ ]:
tf.reset_default_graph()
# We set the constant operators that will store the values of the 
# feature and response variable

X = tf.placeholder("float")
Y = tf.placeholder("float")

In [ ]:
# The weights and bias are defined as variables. Notice that the definition is
# different to the one used above for the features and the target variable
# W and b will be initialized to zero.  By default, Variables are trainable.

W = tf.Variable(0.0, name="weight")
b = tf.Variable(0.0, name="bias")

In [ ]:
# We define the linear model y = wx+b
# Notice that there is only one reponse variable and only one weight 

predicted = tf.add(tf.multiply(X, W), b)

In [ ]:
# The loss function to use will be the mean squared error
loss = tf.reduce_sum(tf.pow(predicted-Y, 2))/(train_n_samples)

In [ ]:
# We define the parameters of the learning process 
learning_rate = 0.05

In [ ]:
# The optimizer is the gradient descent
# It uses as input parameter the learning rate and the loss function
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)


In [ ]:
# Before starting the tensorflow session we need to initialize the parameters
# We init tensorflow variables

#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
init = tf.global_variables_initializer()

# Number of iterations for the learning algorithm
training_epochs = 1000
display_step = 50
perm = np.random.permutation(train_n_samples)


There are three classes of gradient descent methods that WILL BE COVERED IN THE NEXT LECTURE

- Batch gradient descent

- Stochastic gradient descent 

- Mini-batch gradient descent 


Below there is an example of Mini-batch gradient descent implemented with the classical gradient decent minimizer method.

After the model has been learned we plot the predictions in the test set. Uncomment the line where the prediction for the scikit-learn model is plotted in order to compare the prediction of the two algorithms. 

In [ ]:
mini_batch_size = 6
n_batch = train_n_samples // mini_batch_size + (train_n_samples % mini_batch_size != 0)
error = []

# Start training
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:

    # Run the initializer
    sess.run(init)

    # Fit the training data in the batches.
    for epoch in range(training_epochs):
        i_batch = (epoch % n_batch)*mini_batch_size
        batch = train_sel_feature[i_batch:i_batch+mini_batch_size], train_target[i_batch:i_batch+mini_batch_size]
        sess.run(optimizer, feed_dict={X: batch[0], Y: batch[1]})
        
        #for (x, y) in zip(train_sel_feature[perm,:], train_target[perm]):
        #    sess.run(optimizer, feed_dict={X: x, Y: y})

        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            cost = sess.run(loss, feed_dict={X: train_sel_feature, Y: train_target})
            error.append(cost)        
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(cost),  "W=", sess.run(W), "b=", sess.run(b))
                  
       
   
    training_loss = sess.run(loss, feed_dict={X: train_sel_feature , Y: train_target})
    print("Training loss=", training_loss, "W=", sess.run(W), "b=", sess.run(b), '\n')
   
    
    # Visualization of the prediction for the test data
    plt.plot(test_sel_feature, test_target, 'b.', label='Original data')
    plt.plot(test_sel_feature, sess.run(W) * test_sel_feature + sess.run(b),'om', label='tensorflow LR')
    #plt.plot(test_sel_feature,predicted_test_target,'vr', label='Scikit-learn LR')
    plt.xlabel('Voice feature')
    plt.ylabel('Response variable ')

    plt.legend()
    plt.show()
    
 

# Exercise 2 

a) Use tensorflow to learn the model y = a*x^2 + c*x + b where are a,b, and c are the parameters of a linear model.

b) Train the model using two different stochastic gradient descent strategies and the Momentum optimizer



# Exercise 3

a) Use tensorflow to learn the model y = ax + c*z + b where are a,b, and c are the parameters of a linear model and x and z are the two most 
informative features among the 16 variables. 


b) Train the model using the batch gradient decent and the Adam classifier.


# Learning the parameters of a Logistic Regression Classifier with tensorflow

We will consider now a classification problem with 16 features


For this problem we will learn the parameters of a logistic regression classifier

In [ ]:
# As a first step we create the class variable (label)
# binarizing the response variable we had used for regression
scaler = StandardScaler()
label = binarize(scaler.fit(target).transform(target))

In [ ]:
# The class variable is split into train and test sets
train_label = label[::2]
test_label = label[1::2]

We define the variables and constants of the tensorflow session.

In [ ]:
tf.reset_default_graph()


# The input will be a matrix with 16 columns (one for each feature)
x = tf.placeholder(tf.float32, [None, 16])

# Class binary variable
t = tf.placeholder(tf.float32, [None, 1])

# There will be 16 weights, one corresponding to each feature 
w = tf.Variable(tf.zeros([16, 1]))

# This is the intersect, usually also denoted as b
b = tf.Variable(tf.zeros([1]))


We define the elements of the logistic regression model

In [ ]:
# Linear model, matrix multiplication of the variables and the weight
linear_model = tf.matmul(x, w) + b

# The probability is computed combining the linear model and the sigmoid function
LR_prob = tf.sigmoid(linear_model)

We defined the loss function and the accuracy measures used to evaluate the classifier

In [ ]:
# The loss function will be maximum likelihood estimation for the logistic regression model

loss = -tf.reduce_sum(t*tf.log(LR_prob) + (1-t)*tf.log(1-LR_prob))

In [ ]:
# We will find the parameters of the model (the weights)
# by minimizing the loglikelihood with the AdapOptimizer

train_step = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
# Whether, for a given instance, the prediction is correct
correct_prediction = tf.equal(tf.sign(LR_prob-0.5), tf.sign(LR_prob-0.5))

# The accuracy, number of correct predictions divided by the number of examples
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
#sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))

# Initialization of the global variables
init = tf.global_variables_initializer()

In [ ]:
#Auxiliary data for training the model

train_x = train_features
train_t = train_label

In [ ]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
sess.run(init)
i = 0
for _ in range(20000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 500 == 0:
        loss_val, acc_val = sess.run(
            [loss, accuracy], feed_dict={x:train_x, t:train_t})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))

In [ ]:
training_epochs = 2000

    # Fit all training data
for epoch in range(training_epochs):   
    
        # Run the initializer
        sess.run(init)            
        sess.run(train_step, feed_dict={x:train_features, t:train_label})      
        
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:train_features, t:train_label})
            print ('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))
            

# Exercise 4

a) Use tensorflow to learn the linear regression  model y = WX + b where X is a vector of the 16 variables of
the problem (therefore this is a linear model of 16 variables). 

b) Train the model using the stochastic gradient decent and the Momentum classifier.

We download the yacht_hydrodynamics dataset from https://archive.ics.uci.edu/ml/datasets/Yacht+Hydrodynamics

The goal of this dataset is the prediction of residuary resistance of sailing yachts from a number of features.  Essential inputs include the basic hull dimensions and the boat velocity. 

This can be approached as a regression pro blem

In [ ]:

# https://archive.ics.uci.edu/ml/datasets/Yacht+Hydrodynamics
data = np.loadtxt('yacht_hydrodynamics.data')

# The Pandas dataframe is created
df = pd.DataFrame(data,columns=['Long. position', 'Prismatic coef.', 'LD ratio', 'BD ratio', 'LB ratio', 'Froude numb.', 'Resistance' ])


In [ ]:
# Visualization of the dataset
sns.pairplot(df)
plt.show()

In [ ]:
features = data[:,:6]
target = data[:,6]

# We split the data into two sets, training and test

train_features = features[::2,:]
train_target = target[::2]
train_n_samples = train_features.shape[0]

test_features = features[1::2,:]
test_target = target[1::2]
test_n_samples = test_features.shape[0]

feature_selector = SelectKBest(f_regression, k=1)
feature_selector.fit(train_features, train_target)

print(feature_selector.get_support())

train_sel_feature = feature_selector.transform(train_features)
test_sel_feature = feature_selector.transform(test_features)


# Exercise 5

a) Use tensorflow to learn the model y = wx + b where w is the parameter of a linear model of the  yacht_hydrodynamics dataset

b) Train the model using two different stochastic gradient descent methods and the Momentum optimizer

